In [1]:
# We need to join the upper directory in order to access the local modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import itertools
import json
import logging

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

In [3]:
import numpy as np
import pandas as pd
import scipy

In [16]:
import geodesy.geodesy as geodesy
import ltess.ltess as ltess
import mlat.nlls as nlls
from pytdoa import correct_fo
from spec_load import spec_load
import tdoa.tdoa as tdoa

In [5]:
cfg_file = "../.config/config.json"
with open(cfg_file) as f:
    config = json.load(f)

In [6]:
# Load configuration
# Unknown transmitter
fUS_MHz = config["transmitters"]["unknown"]["freq"]
fUS = fUS_MHz * 1e6

# Reference transmitter
fRS_MHz = config["transmitters"]["reference"]["freq"]
fRS = fRS_MHz * 1e6
rs_lat = config["transmitters"]["reference"]["coord"][0]
rs_lon = config["transmitters"]["reference"]["coord"][1]
rs_alt = config["transmitters"]["reference"]["height"]
rs_llh = np.array([rs_lat, rs_lon, rs_alt]).reshape(1, 3)

# Sensor configurations
sr_tdoa = config["config"]["sample_rate"]
sr_ltess = config["config"]["sample_rate_ltess"]
sensors = pd.DataFrame(config["sensors"])
sensors[["latitude", "longitude"]] = sensors.coordinates.to_list()
sensors = sensors.drop(["coordinates"], axis=1)
directory = config["config"]["folder"]
filenum = config["config"]["filenum"]
NUM_SENSORS = len(sensors)

# TDOA estimation
corr_type = config["config"].get("corr_type", "dphase")
interpol = config["config"].get("interpol", 1)
bw_rs = config["transmitters"]["reference"].get("bw", sr_tdoa)
bw_us = config["transmitters"]["unknown"].get("bw", sr_tdoa)

# Design the filter taps for all chunks
taps_rs, taps_us = None, None
if bw_rs < sr_tdoa:
    taps_rs = tdoa.design_filt(bw_rs, sr_tdoa)
    logger.info(f"Filter for reference signal of bandwidth {bw_rs:.2f} created")

if bw_us < sr_tdoa:
    taps_us = tdoa.design_filt(bw_us, sr_tdoa)
    logger.info(f"Filter for targe signal of bandwidth {bw_us:.2f} created")

# Correct the drift of the RTL-SDRs (requires knowing the drift in PPM)
correct = config["config"]["correct"]

# Return the accurate position from NLLS
method = config["config"].get("method", "linear")

In [11]:
selected_list = ["rack_1", "madrid_city_1", "mad_princesa"]
sensorsf = sensors.loc[sensors['name'].isin(selected_list)].copy()

In [8]:
def get_samples_per_sensor(sensors):
    samples = {}
    for (i, sensor) in sensors.iterrows():
        # Computing the distance to the Ref tX
        sensor_llh = np.array(
            [sensor["latitude"], sensor["longitude"], sensor["height"]]
        ).reshape(1, 3)
        dist = geodesy.dist3fromllh(rs_llh, sensor_llh)
        sensors.at[i, "ref_dist"] = dist

        # Compute ECEF coordinates per sensor
        ecef = geodesy.llh2ecef(sensor_llh).squeeze()
        sensors.at[i, "x"] = ecef[0]
        sensors.at[i, "y"] = ecef[1]
        sensors.at[i, "z"] = ecef[2]

        # Loading IQ data
        sname = sensor["name"]
        fname_tdoa = f"{directory}/{sname}/E{filenum}-{int(fRS_MHz)}_{int(fUS_MHz)}-localization.dat"
        fname_ltess = f"{directory}/{sname}/E{filenum}-ltess.dat"

        tdoa_iq = spec_load(fname_tdoa)
        ltess_iq = spec_load(fname_ltess)

        if correct:
            # Estimate Clock drift using the LTESS-Track tool
            (PPM, _, _) = ltess.ltess(ltess_iq, resample_factor=60)
            # Clock correction
            samples[sname] = correct_fo(tdoa_iq, PPM, fRS, fUS, samplingRate=sr_tdoa)
        else:
            samples[sname] = tdoa_iq

    return (samples)

In [12]:
samples = get_samples_per_sensor(sensorsf)

In [13]:
# Get combinations and compute TDOAs per pair
combinations = itertools.combinations(np.arange(len(sensorsf)), 2)

In [14]:
def compute_tdoa_combinations(sensors, samples, combinations):
    combination_list = np.empty((0, 2), dtype=int)
    tdoa_list = np.empty(0)

    for combination in combinations:
        i, j = combination[0], combination[1]

        combination_list = np.vstack(
            (combination_list, [combination[0], combination[1]])
        )

        name_i = sensors.iloc[i]["name"]
        name_j = sensors.iloc[j]["name"]
        rx_diff = sensors.iloc[i]["ref_dist"] - sensors.iloc[j]["ref_dist"]

        tdoa_ij = tdoa.tdoa(
            samples[name_i],
            samples[name_j],
            rx_diff,
            interpol=interpol,
            corr_type=corr_type,
            taps_rs=taps_rs,
            taps_us=taps_us,
        )
        tdoa_list = np.append(tdoa_list, [tdoa_ij["tdoa_m_i"]])

    return (combination_list, tdoa_list)

In [15]:
(combination_list, tdoa_list) = compute_tdoa_combinations(sensorsf, samples, combinations)

In [31]:
sensors_llh = sensorsf[["latitude", "longitude", "height"]].to_numpy()
X0 = np.mean(sensors_llh, axis=0)
altitude = X0[2]

sensors_ecef = geodesy.llh2ecef(
    sensors[["latitude", "longitude", "height"]].to_numpy()
)
sensors_mean = np.mean(sensors_ecef, axis=0)

optimfun = lambda X: nlls.nlls_llh(
    X, altitude, sensors_ecef - sensors_mean, sensors_mean, tdoa_list, combinations
)


,name,height,serial,id,latitude,longitude,ref_dist,x,y,z
0,mad_princesa,600,202481589193614,6,40.43515,-3.6742,1828.915368,4.851972e+06,-311569.067234,4.115271e+06
2,madrid_city_1,600,202481596393530,2,40.39970,-3.7067,4289.580750,4.854342e+06,-314486.224278,4.112273e+06
3,rack_1,600,202481593109723,0,40.33700,-3.7704,12942.622238,4.858489e+06,-320179.527788,4.106968e+06


In [32]:
tdoa_list

array([ -2420.38077089, -11058.80737696,  -8653.0414884 ])